In [19]:
from __future__ import unicode_literals

import requests
import elasticsearch
import elasticsearch.helpers
import elasticsearch2
import elasticsearch2.helpers

In [56]:
import json
import nose.tools

nose.tools.assert_dict_equal.im_self.maxDiff = None

old_client = elasticsearch2.Elasticsearch('http://localhost:9200')
new_client = elasticsearch.Elasticsearch('http://localhost:9201')

def compare_indices(type_, massage_old=lambda x: x, massage_new=lambda x: x, new_index=None, query=None, warn=False):
    query = query or {}

    old_count = old_client.count(index='website', doc_type=type_, body=query)['count']
    new_count = new_client.count(index='test-index-{}s'.format(new_index or type_), body=query)['count']

    total = old_count

    if old_count != new_count:
        if warn:
            print('Old Index: {}. New Index: {}'.format(old_count, new_count))
        else:
            raise Exception('Old Index: {}. New Index: {}'.format(old_count, new_count))
        
    percent, count, size, total = 0, 0, 100, old_count
    
    old_scanner = elasticsearch2.helpers.scan(old_client, query=dict(query, sort=['id']), index='website', doc_type=type_, preserve_order=True)
    new_scanner = elasticsearch.helpers.scan(new_client, query=dict(query, sort=['_id']), index='test-index-{}s'.format(new_index or type_), preserve_order=True)
    
    for old_doc, new_doc in zip(old_scanner, new_scanner):
        old_source, new_source = old_doc['_source'], new_doc['_source']
        new_source['id'] = new_doc['_id']

        nose.tools.assert_dict_equal(massage_old(old_source), massage_new(new_source))
        
        count += 1
        if percent + 10 < (count*100/total):
            percent = (count*100/total)
            print('{}% done ({}/{})'.format(percent, count, total))
    
    print('{} LGTM!'.format(type_))

In [174]:
import re

def first(it):
    return next(x for x in it if x is not None)

def clean(string):
    if isinstance(string, list):
        return [clean(x) for x in string]
    
    if isinstance(string, dict):
        return {k: clean(v) for k, v in string.items()}
    
    if not isinstance(string, basestring):
        return string
    
    if string.startswith('"') and string.endswith('"'):
        return json.loads(string)

    return string

def massage_old_user(user):
    if (user.get('social') or {}).get('profileWebsites'):
        user['social']['profileWebsites'] = json.loads(user['social']['profileWebsites'])
        
    user = clean(user)

    if user.get('all_schools'):
        user['all_schools'] = sorted(user['all_schools'], reverse=True)

    if user.get('all_jobs'):
        user['all_jobs'] = sorted(user['all_jobs'], reverse=True)

    return user


def massage_new_user(user):
    if user.get('all_schools'):
        user['all_schools'] = sorted(user['all_schools'], reverse=True)
        
    if user.get('all_jobs'):
        user['all_jobs'] = sorted(set(user['all_jobs']), reverse=True)
        
    return user

compare_indices('user', massage_old_user, massage_new_user)

0% done (500/116060)
0% done (1000/116060)
1% done (1500/116060)
1% done (2000/116060)
2% done (2500/116060)
2% done (3000/116060)
3% done (3500/116060)
3% done (4000/116060)
3% done (4500/116060)
4% done (5000/116060)
4% done (5500/116060)
5% done (6000/116060)
5% done (6500/116060)
6% done (7000/116060)
6% done (7500/116060)
6% done (8000/116060)
7% done (8500/116060)
7% done (9000/116060)
8% done (9500/116060)
8% done (10000/116060)
9% done (10500/116060)
9% done (11000/116060)
9% done (11500/116060)
10% done (12000/116060)
10% done (12500/116060)
11% done (13000/116060)
11% done (13500/116060)
12% done (14000/116060)
12% done (14500/116060)
12% done (15000/116060)
13% done (15500/116060)
13% done (16000/116060)
14% done (16500/116060)
14% done (17000/116060)
15% done (17500/116060)
15% done (18000/116060)
15% done (18500/116060)
16% done (19000/116060)
16% done (19500/116060)
17% done (20000/116060)
17% done (20500/116060)
18% done (21000/116060)
18% done (21500/116060)
18% done (2

In [197]:
def massage_old_file(doc):
    del doc['is_retracted']
    return doc

def massage_new_file(doc):
    del doc['is_retracted']
    return doc

compare_indices('file', massage_old_file, massage_new_file)

0% done (1000/284111)
0% done (2000/284111)
1% done (3000/284111)
1% done (4000/284111)
1% done (5000/284111)
2% done (6000/284111)
2% done (7000/284111)
2% done (8000/284111)
3% done (9000/284111)
3% done (10000/284111)
3% done (11000/284111)
4% done (12000/284111)
4% done (13000/284111)
4% done (14000/284111)
5% done (15000/284111)
5% done (16000/284111)
5% done (17000/284111)
6% done (18000/284111)
6% done (19000/284111)
7% done (20000/284111)
7% done (21000/284111)
7% done (22000/284111)
8% done (23000/284111)
8% done (24000/284111)
8% done (25000/284111)
9% done (26000/284111)
9% done (27000/284111)
9% done (28000/284111)
10% done (29000/284111)
10% done (30000/284111)
10% done (31000/284111)
11% done (32000/284111)
11% done (33000/284111)
11% done (34000/284111)
12% done (35000/284111)
12% done (36000/284111)
13% done (37000/284111)
13% done (38000/284111)
13% done (39000/284111)
14% done (40000/284111)
14% done (41000/284111)
14% done (42000/284111)
15% done (43000/284111)
15% d

In [204]:
compare_indices('institution')

institution LGTM!


In [18]:
def massage_old_project(doc):
    if not doc['affiliated_institutions']:
        doc['affiliated_institutions'] = []
        
    doc['is_pending_embargo'] = json.loads(doc['is_pending_embargo'])
    doc['wikis'] = {
        x.keys()[0]: x.values()[0]
        for x in (doc['wikis'] or [])
    }
    
    return doc

compare_indices('project', massage_old=massage_old_project, new_index='nodes-project', warn=True)

6% done (10000/143394)
13% done (20000/143394)
20% done (30000/143394)
27% done (40000/143394)
34% done (50000/143394)
41% done (60000/143394)
48% done (70000/143394)
55% done (80000/143394)
62% done (90000/143394)
69% done (100000/143394)
76% done (110000/143394)
83% done (120000/143394)
90% done (130000/143394)
97% done (140000/143394)
project LGTM!


In [27]:
def massage_old_component(doc):
    if not doc['affiliated_institutions']:
        doc['affiliated_institutions'] = []
        
    doc['is_pending_embargo'] = json.loads(doc['is_pending_embargo'])
    doc['wikis'] = {
        x.keys()[0]: x.values()[0]
        for x in (doc['wikis'] or [])
    }
    del doc['license']
    return doc

def massage_new_component(doc):
    del doc['license']
    return doc

compare_indices(
    'component',
    massage_old=massage_old_component,
    massage_new=massage_new_component,
    new_index='nodes-component',
    warn=True
)

28% done (10000/35684)
56% done (20000/35684)
84% done (30000/35684)
component LGTM!


In [32]:
def massage_old_preprint(doc):
    if not doc['affiliated_institutions']:
        doc['affiliated_institutions'] = []
    doc['is_pending_embargo'] = json.loads(doc['is_pending_embargo'])
    doc['wikis'] = {
        x.keys()[0]: x.values()[0]
        for x in (doc['wikis'] or [])
    }
    del doc['license']
    return doc

def massage_new_preprint(doc):
    del doc['license']
    return doc

compare_indices(
    'preprint',
    massage_old=massage_old_preprint,
    massage_new=massage_new_preprint,
    new_index='nodes-preprint',
    warn=True
)

preprint LGTM!


In [57]:
def massage_old_registration(doc):
    if not doc['affiliated_institutions']:
        doc['affiliated_institutions'] = []
    if not doc['is_pending_retraction']:
        doc['is_pending_retraction'] = False
    if not doc['is_pending_registration']:
        doc['is_pending_registration'] = False
    if not doc['is_retracted']:
        doc['is_retracted'] = False
    doc['is_pending_embargo'] = json.loads(doc['is_pending_embargo'] or 'false')
    doc['wikis'] = {
        x.keys()[0]: x.values()[0]
        for x in (doc['wikis'] or [])
    }
    del doc['wikis']
    del doc['license']
    del doc['is_retracted']
    del doc['is_pending_registration']
    return doc

def massage_new_registration(doc):
    del doc['wikis']
    del doc['license']
    del doc['is_retracted']
    del doc['is_pending_registration']
    return doc

compare_indices(
    'registration',
    massage_old=massage_old_registration,
    massage_new=massage_new_registration,
    new_index='nodes-registration',
    warn=True
)

11% done (531/4822)
22% done (1061/4822)
33% done (1592/4822)
44% done (2122/4822)
55% done (2653/4822)
66% done (3183/4822)
77% done (3713/4822)
88% done (4244/4822)
99% done (4774/4822)
registration LGTM!


In [37]:
%debug

> /Users/chrisseto/.pyenv/versions/2.7.15/lib/python2.7/json/decoder.py(364)decode()
    362 
    363         """
--> 364         obj, end = self.raw_decode(s, idx=_w(s, 0).end())
    365         end = _w(s, end).end()
    366         if end != len(s):

ipdb> u
> /Users/chrisseto/.pyenv/versions/2.7.15/lib/python2.7/json/__init__.py(339)loads()
    337             parse_int is None and parse_float is None and
    338             parse_constant is None and object_pairs_hook is None and not kw):
--> 339         return _default_decoder.decode(s)
    340     if cls is None:
    341         cls = JSONDecoder

ipdb> u
> <ipython-input-32-218acad4f3db>(4)massage_old_preprint()
      2     if not doc['affiliated_institutions']:
      3         doc['affiliated_institutions'] = []
----> 4     doc['is_pending_embargo'] = json.loads(doc['is_pending_embargo'])
      5     doc['wikis'] = {
      6         x.keys()[0]: x.values()[0]

ipdb> doc['is_pending_embargo']
ipdb> doc
{u'contributors': [{u'url

ipdb> doc['_id']
*** KeyError: '_id'
ipdb> doc
{u'contributors': [{u'url': u'/549ah/', u'fullname': u'\u26c8  Amanda \u2b50\ufe0f Liscouski \u270d'}, {u'url': u'/aypt8/', u'fullname': u'Amanda J. Liscouscous'}, {u'url': u'/my5xu/', u'fullname': u'Amanda Jo Liscouski'}], u'extra_search_terms': u'', u'normalized_title': u'Different Title Again', u'registered_date': u'2017-05-12T17:48:45.012364+00:00', u'embargo_end_date': None, u'id': u'23dyr', u'category': u'registration', u'title': u'Different Title Again', u'parent_id': None, u'is_pending_registration': None, u'is_pending_embargo': None, u'is_registration': True, u'preprint_url': None, u'boost': 1, u'public': True, u'description': u'0981239763485278127361039843 gfhdsgdsfgsfdgdsfgdsfgdsfg', u'tags': [u'trs', u'fdg', u'dfg', u'mhf', u'ghg'], u'is_pending_retraction': None, u'affiliated_institutions': [], u'is_retracted': None, u'license': {u'id': u'CC0', u'text': u'Statement of Purpose\n\nThe laws of most jurisdictions throughout the wo